In [1]:
import xmltodict
import pandas as pd
import regex as re
from datetime import datetime
import os

In [2]:
from difflib import SequenceMatcher

In [3]:
def read_mxliff_file(file_path):
    with open(file_path, encoding='utf-8') as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
    return data_dict

In [4]:
def extract_data(data_dict):
    list_groups = data_dict['xliff']['file']['body']['group']
    data_for_pd = [(i['@id'], i['@m:para-id'], i['trans-unit']) for i in list_groups]
    df = pd.DataFrame(data_for_pd, columns=['p_id', 'para_id', 'trans_unit'])
    return df

In [5]:
def timestamp_to_human_readable(timestamp_ms):
    timestamp_ms = int(timestamp_ms)
    timestamp = timestamp_ms / 1000  # Convert from milliseconds to seconds
    human_readable_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    return human_readable_time

In [6]:
input_file_path = 'Mark-ar-ar_bh-T.mxliff'

In [7]:
df = extract_data(read_mxliff_file(input_file_path))
df['SOURCE'] = df.trans_unit.apply(lambda x: x['source'])
df['TARGET'] = df.trans_unit.apply(lambda x: x['target'])
df['GROSS_SCORE'] = df.trans_unit.apply(lambda x: float(x['@m:gross-score']))
df['M_SCORE'] = df.trans_unit.apply(lambda x: float(x['@m:score']))
df['MODIFIED_AT'] = df.trans_unit.apply(lambda x: x['@m:modified-at'])
df['MODIFIED_AT'] = df['MODIFIED_AT'].apply(lambda x: timestamp_to_human_readable(x))
df['LOCKED'] = df.trans_unit.apply(lambda x: x['@m:locked'])
df['ALT_MATCH_QUALITY'] = df.trans_unit.apply(lambda x: round(float(x['alt-trans'][1]['@match-quality']), 2))
df['ALT_TARGET'] = df.trans_unit.apply(lambda x: x['alt-trans'][1]['target'])
df['CONFIRMED'] = df.trans_unit.apply(lambda x: x['@m:confirmed'])


In [8]:
test = df.sort_values(['MODIFIED_AT'], ascending=False).head(30)

In [9]:
for i in zip(test.SOURCE, test.TARGET):
    print(i[0])
    print(i[1])
    print('--------------')

\h الإنجيل كما دونه مرقس
\h الإنجيل مثل ما كتبه مرقس
--------------
\v 17 فَإِنَّ هِيرُودُسَ هَذَا كَانَ قَدْ أَرْسَلَ وَقَبَضَ عَلَى يُوحَنَّا وَقَيَّدَهُ فِي السِّجْنِ.
\v 17 لأن هِيرُودُسُ هذا كان طرش على يوحنا حق يجودونه ويجبسونه.
--------------
وَذلِكَ مِنْ أَجْلِ هِيرُودِيَّا الَّتِي تَزَوَّجَهَا هِيِرُودُسُ وَهِيَ زَوْجَةُ أَخِيهِ فِيلِبُّسَ.
وهالشيء لأن هِيِرُودُسُ بغى يتزوج هِيرُودِيَّا وهي كات مَرْت أخوه فِيلِبُّسَ.
--------------
\s1 يسوع يطعم خمسة آلاف
\s1 يسوع يْأَكِّلْ خمسة آلاف
--------------
فَأَخَذَ يَتَحَيَّنُ تَسْلِيمَهُ فِي فُرْصَةٍ مُنَاسِبَةٍ.
فقام يفكر شلون يسلمه لما تجي الفرصة المناسبة.
--------------
\v 31 ثُمَّ غَادَرَ يَسُوعُ نَوَاحِي صُورَ وَعَادَ إِلَى بُحَيْرَةِ الْجَلِيلِ، مُرُوراً بِصَيْدَا وَعَبْرَ حُدُودِ الْمُدُنِ الْعَشْرِ.
\v 31 بعدين طلع يسوع من نواحي صور ورجع إلى بحر الجليل، مار بصيدا وعبر حدود المداين العشر.
--------------
\v 1 ثُمَّ أَخَذَ يُعَلِّمُ ثَانِيَةً عِنْدَ شَاطِئِ الْبُحَيْرَةِ، وَقَدِ احْتَشَدَ حَوْلَهُ جَمْعٌ كَبِيرٌ، حَتَّى إِنَّهُ 

In [10]:
set(''.join(df['SOURCE'])) - set(''.join(df['TARGET']))

{'ٌ', 'ٍ', '\u2006', '–'}

In [11]:
df[df['TARGET'].apply(lambda x: 'ـ' in x)]

,p_id,para_id,trans_unit,SOURCE,TARGET,GROSS_SCORE,M_SCORE,MODIFIED_AT,LOCKED,ALT_MATCH_QUALITY,ALT_TARGET,CONFIRMED
683,683,560,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:683', '@xml:spa...",وَبَيْنَمَا كَانَ خَارِجاً مِنْ أَرِيحَا، وَمَ...,ولما كان طالع من أريحا، ويا تلاميذه وجماعة كبي...,0.99,0.99,2023-08-04 23:43:27,false,0.99,ولما كان طالع من أريحا، ويا تلاميذه وجماعة كبي...,1


In [12]:
df['SOURCE'].iloc[683]

'وَبَيْنَمَا كَانَ خَارِجاً مِنْ أَرِيحَا، وَمَعَهُ تَلامِيذُهُ وَجَمْعٌ كَبِيرٌ، كَانَ بَارْتِيمَاوُسُ الأَعْمَى\u2006–\u2006ابْنُ تِيمَاوُسَ، جَالِساً عَلَى جَانِبِ الطَّرِيقِ يَسْتَعْطِي.'

In [13]:
diacritics = '''
َ
ً
ُ
ٌ
ْ
ٍ
ْ
ّ
'''.strip().split()
diacritics = set(diacritics)

In [14]:
diacritics

{'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ّ', 'ْ'}

In [15]:
pat_diacritics = ''.join(diacritics)
pat_diacritics = fr"[{pat_diacritics}]"

In [16]:
pat_diacritics =  re.compile(pat_diacritics)

In [17]:
df['TARGET'] = df['TARGET'].apply(lambda x: re.sub(pat_diacritics, '', x))
df['SOURCE'] = df['SOURCE'].apply(lambda x: re.sub(pat_diacritics, '', x))

In [18]:
# Utility function to compute similarity
def similar(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

In [21]:
df['SIM_SCORE'] = df.apply(lambda x: similar(x['SOURCE'], x['TARGET']), axis=1)

In [27]:
test = df.sort_values(['SIM_SCORE'])

In [29]:
for i in zip(test['SOURCE'].head(30), test['TARGET'].head(30), test['SIM_SCORE'].head(30)):
    print(i[0])
    print(i[1])
    print(i[2])
    print('----------')

إِنهم سيهابون ابنِي!
أكيد راح يخافوا ولدي!
0.34146341463414637
----------
\ip وينتهي هذا الإنجيل إلى الحديث عن نهاية الزمان وما سيحدث عند رجوع المسيح ثم يسرد الأحداث المتعلقة بآلام المسيح وموته وقيامته وصعوده إلى المجد، ويؤكد على مساندة المسيح لتلاميذه فيما هم ينشرون البشارة في العالم أجمع.
\ip وينتهي هذا الإنجيل عند الكلام عن نهاية الزمان وويش اللي راح يصير لما يرجع المسيح ويحچي الأحداث اللي تخص آلام المسيح وموته وقيامته وصعوده إلى المجد، ويأكد على مساندة المسيح لتلاميذه وهم ينشروا البشارة في العالم كله.
0.3448275862068966
----------
ثم صرفهم،
وبعدين مشاهم،
0.36363636363636365
----------
وأيضا:
وبعد:
0.36363636363636365
----------
\v 26 وقد عانتِ الكثِير مِن الألمِ على أيدِي أطِباء كثِيرِين، وأنفقت فِي سبِيلِ عِلاجِها كل ما تملِك، فلم تجنِ أية فائِدة، بل بِالأحرى ازدادت حالتها سوءا.
\v 26 وقاست واجد من الألم وشافت أطباء واجد، وصرفت كل الفلوس اللي تملكها على العلاج، وما حصلت فايدة، بالعكس زادت حالتها بلى.
0.37545126353790614
----------
فلأجلِ هذا جِئت.»
هذا الشيء اللي جيت حقه.»
0.39024

In [32]:
test['LEN_DIFF'] = test.apply(lambda x: abs(len(x['SOURCE'].split())  - len(x['TARGET'].split())), axis=1)

In [34]:
test['LEN_DIFF'].describe()

count    1126.000000
mean        0.582593
std         0.879302
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         7.000000
Name: LEN_DIFF, dtype: float64

,p_id,para_id,trans_unit,SOURCE,TARGET,GROSS_SCORE,M_SCORE,MODIFIED_AT,LOCKED,ALT_MATCH_QUALITY,ALT_TARGET,CONFIRMED,SIM_SCORE,LEN_DIFF
656,656,538,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:656', '@xml:spa...",\v 31 وهناك أولون كثِيرون يصِيرون آخِرِين، وال...,\v 31 وهناك ناس كثيرين هم في الأول يصيروا في ا...,0.96,0.96,2023-08-04 23:43:21,false,0.96,31 وهناك ناس كثيرين هم في الأول يصيروا في الأخ...,1,0.467532,7
1049,1049,836,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:1049', '@xml:sp...",\v 20 وبعدما أوسعوه سخرِية، نزعوا رِداء الأرجو...,\v 20 وبعد ما كثروا من الاستهزاء فيه، شالوا عن...,0.97,0.97,2023-08-04 23:44:49,false,0.97,20 وبعد ما كثروا من الاستهزاء فيه، شالوا عنه ث...,1,0.600000,5
781,781,629,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:781', '@xml:spa...",\v 15 ولكِنه إِذ علِم نِفاقهم قال لهم: «لِماذا...,\v 15 ولكن لأنه كان يعرف إن هذا نفاق منهم فقال...,0.00,0.00,2023-08-04 23:43:48,false,0.56,هل يجوز لينا ندفع الجزية لقيصر أو لا؟ ندفعها أ...,1,0.633333,5
222,222,185,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:222', '@xml:spa...",\v 22 فليس مخفِي إِلا ويكشف، وما كتِم شيء إِلا...,\v 22 فما في شيء مخفي إلا وراح ينكشف، وما في ش...,0.00,0.00,2023-08-04 23:41:37,false,0.00,None,1,0.661290,5
379,379,314,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:379', '@xml:spa...",\v 41 ثم أخذ يسوع الأرغِفة الخمسة والسمكتينِ، ...,\v 41 بعدين أخذ يسوع أقراص الخبز الخمسة والسمچ...,0.98,0.98,2023-08-04 23:42:15,false,0.98,41 بعدين أخذ يسوع أقراص الخبز الخمسة والسمچتين...,1,0.737179,5
49,49,43,"{'@id': '0rgV1fdxzRMUsfOA1_dc5:49', '@xml:spac...",\v 22 فذهِل الحاضِرون مِن تعلِيمِهِ، لأنه كان ...,\v 22 وانهدش الحاضرين من تعليمه، لأنه كان يعلم...,0.97,0.97,2023-08-04 23:45:17,false,0.97,22 وانهدش الحاضرين من تعليمه، لأنه كان يعلمهم ...,1,0.748538,5
